In [20]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2023-04-18 06:50:24,682: Credentials are already in use. The existing account in the session will be replaced.


In [2]:
from PIL import Image
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram


In [3]:
#normlize the intensity of pixel 0 to 1

In [9]:
# Load the image file to embed text in
image_path = 'image8x8.png'
img = Image.open(image_path).convert('L')
pixels = img.load()
width, height = img.size

In [10]:
# Convert the text message to binary format
message = 'Hello'
binary_message = ''.join(format(ord(c), '08b') for c in message)

In [11]:
# Create a two-dimensional grid of qubits, where each qubit corresponds to a pixel in the image
num_qubits = width * height
qc = QuantumCircuit(num_qubits, num_qubits)

In [12]:
# Initialize the qubits to a superposition state
for i in range(num_qubits):
    qc.h(i)

In [13]:
# Initialize the qubits to a superposition state
for i in range(num_qubits):
    qc.h(i)

In [14]:
# Encode the binary message onto the grid using controlled rotations
for i, bit in enumerate(binary_message):
    if bit == '1':
        pixel_index = i * 2  # Encode onto two qubits per pixel
        if pixel_index < num_qubits:
            x = pixel_index % width
            y = pixel_index // width
            for j in range(num_qubits):
                if j // width == y and j != pixel_index:
                    qc.cx(pixel_index, j)
                if j % width == x and j != pixel_index:
                    qc.cx(pixel_index, j)

In [15]:
# Measure the qubits to collapse the quantum state into a classical state
qc.measure(range(num_qubits), range(num_qubits))

In [16]:
# Simulate the circuit and retrieve the measurement outcomes
print("Entering Quantum simulator...")
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1024)
result = job.result()
counts = result.get_counts()
print("Exiting Quantum simulator...")


Entering Quantum simulator...
Exiting Quantum simulator...


In [19]:
qc.draw()

In [17]:
# Convert the measurement outcome back into an image with the text message embedded
for i in range(num_qubits):
    pixel_index = i * 2
    x = pixel_index % width
    y = pixel_index // width
    if counts['0' * num_qubits] == 0 and counts[str(bin(i)[2:]).zfill(num_qubits)] == 1:
        # Apply a white or black color depending on the state of the qubit
        if pixels[x, y] < 128:
            pixels[x, y] = 0
        else:
            pixels[x, y] = 255

In [18]:
# Save the modified image with the embedded message
img.save('embedded_image01.jpg')

In [14]:
import numpy as np

def decode_image(encoded_img, decode_matrix):
    # Extract the message bits from the encoded image
    img_shape = encoded_img.shape
    msg_bits = np.zeros(img_shape, dtype=np.int)
    for i in range(img_shape[0]):
        for j in range(img_shape[1]):
            block = encoded_img[i:i+3, j:j+3]
            msg_bits[i,j] = int(np.sum(decode_matrix * block) < 0)
    
    # Convert the message bits to bytes
    msg_bit_str = ''.join([str(b) for b in msg_bits.flatten()])
    msg_bytes = bytearray(int(msg_bit_str[i:i+8], 2) for i in range(0, len(msg_bit_str), 8))
    
    # Decode the message
    return msg_bytes.decode('utf-8')


In [17]:
import numpy as np
from PIL import Image

# Load the image from the PNG file
img = Image.open('embedded_image.jpg')

# Convert the image to a numpy array
img_array = np.array(img)

# Save the numpy array to an NPY file
np.save('embedded_image.npy', img_array)


In [18]:
# Define the decoding matrix
decode = np.array([[1, 0, -1], [-1, 1, 0], [0, -1, 1]]) / np.sqrt(2)

# Load the encoded image from file
encoded_img = np.load('embedded_image.npy')

# Decode the message
decoded_msg = decode_image(encoded_img, decode)
print("The decoded message is:", decoded_msg)


Traceback (most recent call last):
  Cell In[18], line 8
    decoded_msg = decode_image(encoded_img, decode)
  Cell In[14], line 10 in decode_image
    msg_bits[i,j] = int(np.sum(decode_matrix * block) < 0)
ValueError: operands could not be broadcast together with shapes (3,3) (3,2) 

Use %tb to get the full traceback.


/tmp/ipykernel_118/3670053975.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  msg_bits = np.zeros(img_shape, dtype=np.int)
